In [1]:
!curl https://developer.download.nvidia.com/hpc-sdk/ubuntu/DEB-GPG-KEY-NVIDIA-HPC-SDK | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1626  100  1626    0     0   9036      0 --:--:-- --:--:-- --:--:--  9083


In [2]:
! echo 'deb [signed-by=/usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 /' | sudo tee /etc/apt/sources.list.d/nvhpc.list

deb [signed-by=/usr/share/keyrings/nvidia-hpcsdk-archive-keyring.gpg] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 /


In [3]:
!sudo apt-get update -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64  InRelease [2,126 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64  Packages [22.7 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:13 https://ppa.launchpadcontent.net/u

In [4]:
!sudo apt-get install -y nvhpc-24-5

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  nvhpc-24-5
0 upgraded, 1 newly installed, 0 to remove and 47 not upgraded.
Need to get 4,237 MB of archives.
After this operation, 14.2 GB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64  nvhpc-24-5 24.5 [4,237 MB]
Fetched 4,237 MB in 52s (81.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package nvhpc-24-5.
(Reading database ... 121918 files and director

In [5]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/compilers/bin/nvaccelinfo


CUDA Driver Version:           12020
NVRM version:                  NVIDIA UNIX x86_64 Kernel Module  535.104.05  Sat Aug 19 01:15:15 UTC 2023

Device Number:                 0
Device Name:                   Tesla T4
Device Revision Number:        7.5
Global Memory Size:            15835660288
Number of Multiprocessors:     40
Concurrent Copy and Execution: Yes
Total Constant Memory:         65536
Total Shared Memory per Block: 49152
Registers per Block:           65536
Warp Size:                     32
Maximum Threads per Block:     1024
Maximum Block Dimensions:      1024, 1024, 64
Maximum Grid Dimensions:       2147483647 x 65535 x 65535
Maximum Memory Pitch:          2147483647B
Texture Alignment:             512B
Clock Rate:                    1590 MHz
Execution Timeout:             No
Integrated Device:             No
Can Map Host Memory:           Yes
Compute Mode:                  default
Concurrent Kernels:            Yes
ECC Enabled:                   Yes
Memory Clock Rate: 

In [6]:
%%writefile openAcc_first.c
#include <stdio.h>
#define N 1000
int array[N];
int main() {
#pragma acc parallel loop copy(array[0:N])
   for(int i = 0; i < N; i++) {
      array[i] = 3.0;
   }
   printf("Success!\n");
}


Writing openAcc_first.c


In [8]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/compilers/bin/nvc -acc openAcc_first.c -o openAcc_first

In [13]:
!./openAcc_first

Success!


In [37]:
!./openAcc_first

Success!


In [46]:
%%writefile openacc_array.c
#include <stdio.h>
#define N 10
int array[N];
int main() {
#pragma acc parallel loop copy(array[0:N])
   for(int i = 0; i < N; i++) {
      array[i] = i;
      printf("%d\n",array[i]);
   }
   printf("Success!\n");
}

Writing openacc_array.c


In [47]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/compilers/bin/nvc -acc openacc_array.c -o openacc_array

In [48]:
!./openacc_array

2
7
1
6
3
0
8
5
4
9
Success!


In [12]:
ls-l

total 36
-rwxr-xr-x 1 root root 28208 May 29 05:01 openAcc_first*
-rw-r--r-- 1 root root   194 May 29 04:40 openAcc_first.c
drwxr-xr-x 1 root root  4096 May 24 13:22 sample_data/


In [21]:
%%writefile openacc_second.c
#include<stdio.h>
#include<stdlib.h>

#define N 100

void vecaddgpu(float *restrict r,float *a,float *b,int n){
  #pragma acc kernels loop copyin(a[0:n],b[0:n])copyout(r[0:n])
  for(int i=0;i<n;++i)r[i]=a[i]+b[i];
}
int main(int n,int argc,char* argv[]){
  float * a;/*input vector 1*/
  float * b;/*input vector 2*/
  float * r;/*output vector */
  float * e;/*expected output value*/
  int i,errs;
  a=(float*)malloc(N*sizeof(float));
  b=(float*)malloc(N*sizeof(float));
  r=(float*)malloc(N*sizeof(float));
  e=(float*)malloc(N*sizeof(float));
  for(int i=0;i<n;++i){
    a[i]=(float)(i+1);
    b[i]=(float)(1000*i);
  }
  /*computr on gpu */
  vecaddgpu(r,a,b,N);

  /*compute on the host to compare */
  for(i=0;i<n;++i) e[i]=a[i]+b[i];

  /* comprae the result */
  errs=0;
  for(i=0;i<n;++i){
    if(r[i]!=e[i]){
      ++errs;
    }
  }
  printf("%d errors found\n",errs);
  return errs;
}

Overwriting openacc_second.c


In [22]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/compilers/bin/nvc -acc openacc_second.c -o openacc_second

In [23]:
!./openacc_second

0 errors found


In [35]:
%%writefile openacc_third.c
#include<stdio.h>

int main(){
  int n=100;
  float A[n],B[n];

  //initialze arrays Anad B on the host(cpu)
  for(int i=0;i<n;++i){
    A[i]=i*0.1f;
    B[i]=i*0.1f;
  }
  //transer the data to the gpu
  #pragma acc enter data copyin(A,B)

  //first data region:calculate element wise sum(A+B) On device
  #pragma acc data present(A,B)
  {
    for(int i=0;i<n;++i){
      A[i]=A[i]+B[i];
    }
  }
//second data region :square each element of A on device(using present agian)
#pragma acc data present(A)
{
  for(int i=0;i<n;++i){
      A[i]=A[i] * A[i];
}
}
//transer final result in A back to host
#pragma acc exit data copyout(A)

//print final results(now accesable on the host)
printf("square element of (A+B):\n");
for(int i=0;i<n;++i){
  printf("%f",A[i]);
}
printf("\n");
return 0;
}

Overwriting openacc_third.c


In [36]:
!/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/compilers/bin/nvc -acc openacc_third.c -o openacc_third

In [38]:
!./openacc_third

square element of (A+B):
0.0000000.1000000.2000000.3000000.4000000.5000000.6000000.7000000.8000000.9000001.0000001.1000001.2000001.3000001.4000001.5000001.6000001.7000001.8000001.9000002.0000002.1000002.2000002.3000002.4000002.5000002.6000002.7000002.8000002.9000003.0000003.1000003.2000003.3000003.4000003.5000003.6000003.7000003.8000003.9000004.0000004.1000004.2000004.3000004.4000004.5000004.6000004.7000004.8000004.9000005.0000005.1000005.2000005.3000005.4000005.5000005.6000005.7000005.8000005.9000006.0000006.1000006.2000006.3000006.4000006.5000006.6000006.7000006.8000006.9000007.0000007.1000007.2000007.3000007.4000007.5000007.6000007.7000007.8000007.9000008.0000008.1000008.2000008.3000008.4000018.5000008.6000008.7000008.8000008.9000019.0000009.1000009.2000009.3000009.4000019.5000009.6000009.7000009.8000009.900001


In [43]:
%%writefile cuda_assignment29may.cu
/*for the 'present' data clause code example ,write corresponding cod eincuda in bothe codes file add timing code ,compare performance for 10 lakh and 1 crore elements */
#include <stdio.h>

__global__ void kernel1(float *A, float *B, int n) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if (idx < n) {
    A[idx] = A[idx] + B[idx];
  }
}

__global__ void kernel2(float *A, int n) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if (idx < n) {
    A[idx] = A[idx] * A[idx];
  }
}

int main() {
  int n = 100;
  float *A, *B;
  cudaMalloc((void **)&A, n * sizeof(float));
  cudaMalloc((void **)&B, n * sizeof(float));

  // Initialize arrays A and B on the host (CPU)
  for (int i = 0; i < n; ++i) {
    A[i] = i * 0.1f;
    B[i] = i * 0.1f;
  }

  // Copy data from host to device
  cudaMemcpy(A, A, n * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(B, B, n * sizeof(float), cudaMemcpyHostToDevice);

  // Launch kernel 1: calculate element-wise sum (A+B) on device
  int blockSize = 256;
  int numBlocks = (n + blockSize - 1) / blockSize;
  kernel1<<<numBlocks, blockSize>>>(A, B, n);

  // Launch kernel 2: square each element of A on device
  kernel2<<<numBlocks, blockSize>>>(A, n);

  // Copy final result in A back to host
  cudaMemcpy(A, A, n * sizeof(float), cudaMemcpyDeviceToHost);

  // Print final results (now accessible on the host)
  printf("Square elements of (A+B):\n");
  for (int i = 0; i < n; ++i) {
    printf("%f ", A[i]);
  }
  printf("\n");

  cudaFree(A);
  cudaFree(B);

  return 0;
}

Overwriting cuda_assignment29may.cu


In [49]:
!nvcc cuda_assignment29may.cu -o cuda_assig_29may

In [51]:
!./cuda_assig_29may